# Notebook to fit the models

Here we are fitting the models using pyabc (running the fitting in python)

In [ ]:
 from julia import Julia
jl = Julia(sysimage = "/home/callum/ASF/Fitting/sys_model.so") #loading sys image

In [ ]:
import matplotlib.pyplot as plt

import os
import tempfile
import numpy as np
import scipy as sp


from pyabc import ABCSMC, RV, Distribution, LocalTransition, MedianEpsilon, SingleCoreSampler
from pyabc.visualization import plot_data_callback, plot_kde_2d
from pyabc.populationstrategy import AdaptivePopulationSize
import pyabc
from scipy import stats
import random as rd
from brokenaxes import brokenaxes

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [ ]:
plt.style.use('seaborn-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Fitting ODE model and deciding best!

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/ODE_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/ODE_FIT.jl') #loading files with our model!

In [ ]:
#Loading the four models!
model1 = jl.SIR_ODE.model_1 #Frequency
model2 = jl.SIR_ODE.model_2 #Pure Density
model3 = jl.SIR_ODE.model_3 #Sigmoid Density
model4 = jl.SIR_ODE.model_4 #Sqrt Density

models  =[model1,model2,model3,model4]

In [ ]:
#The observations!
obs = jl.SIR_ODE.observation

#Distance function used for fitting
distance = jl.SIR_ODE.distance


In [ ]:
#Fitting two params P1 - transmission co-efficent, P2 - corpse infectivity modifier
parameter_prior = Distribution(p1=RV("uniform", 0.0, 1.0),p2=RV("uniform", 0.0, 1.0))

#sample params to test functions all work and pre-compile
sample_par = {"p1": 0.142,"p2": 0.006} 

n = 4 #number of models we are looking at!
param_list = [parameter_prior]*n

In [ ]:
param_list = [Distribution(p1=RV("uniform", 0.1, 0.25),p2=RV("uniform", 0.0, 0.01)), Distribution(p1=RV("uniform", 0.0, 0.5),p2=RV("uniform", 0.75, 1.0)),Distribution(p1=RV("uniform", 0.0, 0.5),p2=RV("uniform", 0.75, 1.0)),Distribution(p1=RV("uniform", 0.0, 0.5),p2=RV("uniform", 0.75, 1.0))]

In [ ]:
#running the fitting!
abc= ABCSMC(
    models=model1,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size = AdaptivePopulationSize(100, 0.15,max_population_size = 1000),
    sampler = SingleCoreSampler(),
)


abc_id = abc.new("sqlite:////tmp/mjp.db", obs)
it = 20 #number of generations we want smc to do!
m1 = abc.run(minimum_epsilon=0.5, max_nr_populations=it)


In [ ]:
#saving the fit!
np.savetxt("Posteriors/ODE/L1_params.csv",  m1.get_distribution(t=it-1)[0].values, delimiter =',')
np.savetxt("/home/callum/ASF/Fitting/Posteriors/ODE/L1_weights.csv", m1.get_distribution(t=it-1)[1], delimiter= ',')

# Tau Leaping Homogeneous 

In [ ]:
#The same as above but for M2!

jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HOMO_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HOMO_FIT.jl') #loading files with our model!


In [ ]:
model1_h = jl.SIR_TAU_S.model_1 #Frequency
model2_h = jl.SIR_TAU_S.model_2 #Pure Density
model3_h = jl.SIR_TAU_S.model_3 #Sigmoid Density
model4_h = jl.SIR_TAU_S.model_4 #Sqrt Density

models_h  =[model1_h,model2_h,model3_h,model4_h]

In [ ]:
abc= ABCSMC(
    models=model1_h,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size = AdaptivePopulationSize(100, 0.15,max_population_size = 1000),
    sampler = SingleCoreSampler(),
)

it=10

abc_id = abc.new("sqlite:////tmp/mjp.db", obs)
m1h = abc.run(minimum_epsilon=0.5, max_nr_populations=it)

In [ ]:
np.savetxt("/home/callum/ASF/Fitting/Posteriors/Tau-Homogeneous/L1_params.csv",  m1h.get_distribution(t=it-1)[0].values, delimiter =',')
np.savetxt("/home/callum/ASF/Fitting/Posteriors/Tau-Homogeneous/L1_weights.csv", m1h.get_distribution(t=it-1)[1], delimiter= ',')


# Tau Hetro

In [ ]:
jl.eval('push!(LOAD_PATH, "/home/callum/ASF/Fitting/TAU_HETRO_FIT.jl")')
jl.include('/home/callum/ASF/Fitting/TAU_HETRO_FIT.jl') #loading files with our model!
#the network we are using is defined in the Hetro_Fit file!

In [ ]:
model1= jl.SIR_TAU_M.model_1 #Frequency
model4 = jl.SIR_TAU_M.model_4 #Sqrt Density

In [ ]:
#Fitting two params P1 - transmission co-efficent, P2 - corpse infectivity modifier
parameter_prior = Distribution(p1=RV("uniform", 0.0, 1.0),p2=RV("uniform", 0.0, 0.25),p3=RV("uniform", 0.0, 1.0),)

#sample params to test functions all work and pre-compile
sample_par = {"p1": 0.1,"p2": 0.05, "p3": 0.5} 

n = 4 #number of models we are looking at!
param_list = [parameter_prior]*n

In [ ]:
distance(model1_c (sample_par),obs)
distance(model2_c (sample_par),obs)
distance(model3_c (sample_par),obs)
distance(model4_c (sample_par),obs);

In [ ]:
abc = ABCSMC(
    models=model1_c,
    parameter_priors=parameter_prior,
    distance_function=distance,
    population_size =  AdaptivePopulationSize(100, 0.15,max_population_size = 100),
    sampler = SingleCoreSampler(),
)




In [ ]:
abc_id = abc.new("sqlite:////tmp/mjjp.db", obs)


In [ ]:
h1n = abc.run(minimum_epsilon=0.5, max_nr_populations=6)
